Aim: find top  3-5 commuter-heavy train stations

1. Clean data
2. Groupby

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
col_names = ['BOOTH','UNIT','SCP','STATION','LINENAME','DIVISION','DATE','TIME','DESC', 'ENTRIES','EXITS']
df = pd.read_table('../data/may1.txt', sep = ",", header = 0, names = col_names)

In [3]:
df.head()

,BOOTH,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,00:00:00,REGULAR,7035249,2384833
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,04:00:00,REGULAR,7035269,2384840
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,08:00:00,REGULAR,7035292,2384875
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,12:00:00,REGULAR,7035392,2384951
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,16:00:00,REGULAR,7035651,2385020


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206857 entries, 0 to 206856
Data columns (total 11 columns):
BOOTH       206857 non-null object
UNIT        206857 non-null object
SCP         206857 non-null object
STATION     206857 non-null object
LINENAME    206857 non-null object
DIVISION    206857 non-null object
DATE        206857 non-null object
TIME        206857 non-null object
DESC        206857 non-null object
ENTRIES     206857 non-null int64
EXITS       206857 non-null int64
dtypes: int64(2), object(9)
memory usage: 17.4+ MB


In [5]:
df['DATE'] =  pd.to_datetime(df['DATE'], format= "%m/%d/%Y")
df['DATE'].dt.date

0         2019-04-27
1         2019-04-27
2         2019-04-27
3         2019-04-27
4         2019-04-27
             ...    
206852    2019-05-03
206853    2019-05-03
206854    2019-05-03
206855    2019-05-03
206856    2019-05-03
Name: DATE, Length: 206857, dtype: object

In [6]:
df['DAY'] = df['DATE'].dt.dayofweek

In [7]:
df['TIME'][1]

'04:00:00'

In [8]:
df['TIME'] =  pd.to_datetime(df['TIME'], format = "%H:%M:%S")
df['TIME'].dt.time

0         00:00:00
1         04:00:00
2         08:00:00
3         12:00:00
4         16:00:00
            ...   
206852    05:00:00
206853    09:00:00
206854    13:00:00
206855    17:00:00
206856    21:00:00
Name: TIME, Length: 206857, dtype: object

In [9]:
#  The Subunit Channel Position represents the turnstile and the number used may repeat across stations. 
# The UNIT and SCP together is a unique identifier of a turnstile.

In [10]:
df['STATLINE'] = df['STATION'] + '-' + df['LINENAME']

In [11]:
grouped = df.groupby(['STATLINE', 'BOOTH', 'SCP', 'DATE'])
grouped = grouped.apply(lambda _df: _df.sort_values(by=['TIME']))

In [12]:
grouped['entry_diff'] = grouped.ENTRIES.diff()

In [13]:
grouped['exit_diff'] = grouped.EXITS.diff()

In [14]:
grouped

BOOTH  UNIT       SCP    STATION  \
STATLINE    BOOTH SCP      DATE                                                 
1 AV-L      H007  00-00-00 2019-04-27 31049   H007  R248  00-00-00       1 AV   
                                      31050   H007  R248  00-00-00       1 AV   
                                      31051   H007  R248  00-00-00       1 AV   
                                      31052   H007  R248  00-00-00       1 AV   
                                      31053   H007  R248  00-00-00       1 AV   
...                                            ...   ...       ...        ...   
ZEREGA AV-6 R419  00-05-01 2019-05-03 181659  R419  R326  00-05-01  ZEREGA AV   
                                      181660  R419  R326  00-05-01  ZEREGA AV   
                                      181661  R419  R326  00-05-01  ZEREGA AV   
                                      181662  R419  R326  00-05-01  ZEREGA AV   
                                      181663  R419  R326  00-05-01  ZEREGA AV   

                                             LINENAME DIVISION       DATE  \
STATLINE    BOOTH SCP      DATE                                             
1 AV-L      H007  00-00-00 2019-04-27 31049         L      BMT 2019-04-27   
                                      31050         L      BMT 2019-04-27   
                                      31051         L      BMT 2019-04-27   
                                      31052         L      BMT 2019-04-27   
                                      31053         L      BMT 2019-04-27   
...                                               ...      ...        ...   
ZEREGA AV-6 R419  00-05-01 2019-05-03 181659        6      IRT 2019-05-03   
                                      181660        6      IRT 2019-05-03   
                                      181661        6      IRT 2019-05-03   
                                      181662        6      IRT 2019-05-03   
                                      181663        6      IRT 2019-05-03   

                                                            TIME     DESC  \
STATLINE    BOOTH SCP      DATE                                             
1 AV-L      H007  00-00-00 2019-04-27 31049  1900-01-01 00:00:00  REGULAR   
                                      31050  1900-01-01 04:00:00  REGULAR   
                                      31051  1900-01-01 08:00:00  REGULAR   
                                      31052  1900-01-01 12:00:00  REGULAR   
                                      31053  1900-01-01 16:00:00  REGULAR   
...                                                          ...      ...   
ZEREGA AV-6 R419  00-05-01 2019-05-03 181659 1900-01-01 05:00:00  REGULAR   
                                      181660 1900-01-01 09:00:00  REGULAR   
                                      181661 1900-01-01 13:00:00  REGULAR   
                                      181662 1900-01-01 17:00:00  REGULAR   
                                      181663 1900-01-01 21:00:00  REGULAR   

                                               ENTRIES     EXITS  DAY  \
STATLINE    BOOTH SCP      DATE                                         
1 AV-L      H007  00-00-00 2019-04-27 31049   14647394  16373694    5   
                                      31050   14647394  16373709    5   
                                      31051   14647395  16373732    5   
                                      31052   14647395  16373766    5   
                                      31053   14647395  16373800    5   
...                                                ...       ...  ...   
ZEREGA AV-6 R419  00-05-01 2019-05-03 181659        39       145    4   
                                      181660        39       145    4   
                                      181661        39       145    4   
                                      181662        39       145    4   
                                      181663        39       145    4   

                                     

In [15]:

grouped[(grouped.entry_diff < 0)].groupby([grouped.STATLINE, grouped.SCP]).head()

BOOTH  UNIT       SCP    STATION  \
STATLINE    BOOTH SCP      DATE                                                 
1 AV-L      H007  00-03-01 2019-04-27 31184   H007  R248  00-03-01       1 AV   
            H008  01-00-00 2019-04-27 31274   H008  R248  01-00-00       1 AV   
                  01-00-02 2019-04-27 31358   H008  R248  01-00-02       1 AV   
                  01-00-04 2019-04-27 31442   H008  R248  01-00-04       1 AV   
103 ST-1    R170  00-03-00 2019-04-27 145321  R170  R191  00-03-00     103 ST   
...                                            ...   ...       ...        ...   
YORK ST-F   N530  00-00-02 2019-04-27 103518  N530  R301  00-00-02    YORK ST   
ZEREGA AV-6 R419  00-00-00 2019-04-27 181412  R419  R326  00-00-00  ZEREGA AV   
                  00-03-01 2019-04-27 181538  R419  R326  00-03-01  ZEREGA AV   
                  00-05-00 2019-04-27 181580  R419  R326  00-05-00  ZEREGA AV   
                  00-05-01 2019-04-27 181622  R419  R326  00-05-01  ZEREGA AV   

                                             LINENAME DIVISION       DATE  \
STATLINE    BOOTH SCP      DATE                                             
1 AV-L      H007  00-03-01 2019-04-27 31184         L      BMT 2019-04-27   
            H008  01-00-00 2019-04-27 31274         L      BMT 2019-04-27   
                  01-00-02 2019-04-27 31358         L      BMT 2019-04-27   
                  01-00-04 2019-04-27 31442         L      BMT 2019-04-27   
103 ST-1    R170  00-03-00 2019-04-27 145321        1      IRT 2019-04-27   
...                                               ...      ...        ...   
YORK ST-F   N530  00-00-02 2019-04-27 103518        F      IND 2019-04-27   
ZEREGA AV-6 R419  00-00-00 2019-04-27 181412        6      IRT 2019-04-27   
                  00-03-01 2019-04-27 181538        6      IRT 2019-04-27   
                  00-05-00 2019-04-27 181580        6      IRT 2019-04-27   
                  00-05-01 2019-04-27 181622        6      IRT 2019-04-27   

                                                            TIME     DESC  \
STATLINE    BOOTH SCP      DATE                                             
1 AV-L      H007  00-03-01 2019-04-27 31184  1900-01-01 00:00:00  REGULAR   
            H008  01-00-00 2019-04-27 31274  1900-01-01 00:00:00  REGULAR   
                  01-00-02 2019-04-27 31358  1900-01-01 00:00:00  REGULAR   
                  01-00-04 2019-04-27 31442  1900-01-01 00:00:00  REGULAR   
103 ST-1    R170  00-03-00 2019-04-27 145321 1900-01-01 01:00:00  REGULAR   
...                                                          ...      ...   
YORK ST-F   N530  00-00-02 2019-04-27 103518 1900-01-01 01:00:00  REGULAR   
ZEREGA AV-6 R419  00-00-00 2019-04-27 181412 1900-01-01 01:00:00  REGULAR   
                  00-03-01 2019-04-27 181538 1900-01-01 01:00:00  REGULAR   
                  00-05-00 2019-04-27 181580 1900-01-01 01:00:00  REGULAR   
                  00-05-01 2019-04-27 181622 1900-01-01 01:00:00  REGULAR   

                                               ENTRIES    EXITS  DAY  \
STATLINE    BOOTH SCP      DATE                                        
1 AV-L      H007  00-03-01 2019-04-27 31184    2433288  1042318    5   
            H008  01-00-00 2019-04-27 31274     269598  4334607    5   
                  01-00-02 2019-04-27 31358     533699  1236970    5   
                  01-00-04 2019-04-27 31442     198884    23822    5   
103 ST-1    R170  00-03-00 2019-04-27 145321   8281630  4637365    5   
...                                                ...      ...  ...   
YORK ST-F   N530  00-00-02 2019-04-27 103518  11691242  1603033    5   
ZEREGA AV-6 R419  00-00-00 2019-04-27 181412     33025   135035    5   
                  00-03-01 2019-04-27 181538    771861   137746    5   
                  00-05-00 2019-04-27 181580       197        0    5   
                  00-05-01 2019-04-27 181622        39      145    5   

                                                 S

In [16]:
grouped[(grouped.STATLINE == '1 AV-L') & (grouped.BOOTH =='H007') & (grouped.SCP =='00-03-01')]

BOOTH  UNIT       SCP STATION  \
STATLINE BOOTH SCP      DATE                                             
1 AV-L   H007  00-03-01 2019-04-27 31184  H007  R248  00-03-01    1 AV   
                                   31185  H007  R248  00-03-01    1 AV   
                                   31186  H007  R248  00-03-01    1 AV   
                                   31187  H007  R248  00-03-01    1 AV   
                                   31188  H007  R248  00-03-01    1 AV   
                                   31189  H007  R248  00-03-01    1 AV   
                        2019-04-28 31190  H007  R248  00-03-01    1 AV   
                                   31191  H007  R248  00-03-01    1 AV   
                                   31192  H007  R248  00-03-01    1 AV   
                                   31193  H007  R248  00-03-01    1 AV   
                                   31194  H007  R248  00-03-01    1 AV   
                                   31195  H007  R248  00-03-01    1 AV   
                        2019-04-29 31196  H007  R248  00-03-01    1 AV   
                                   31197  H007  R248  00-03-01    1 AV   
                                   31198  H007  R248  00-03-01    1 AV   
                                   31199  H007  R248  00-03-01    1 AV   
                                   31200  H007  R248  00-03-01    1 AV   
                                   31201  H007  R248  00-03-01    1 AV   
                                   31202  H007  R248  00-03-01    1 AV   
                        2019-04-30 31203  H007  R248  00-03-01    1 AV   
                                   31204  H007  R248  00-03-01    1 AV   
                                   31205  H007  R248  00-03-01    1 AV   
                                   31206  H007  R248  00-03-01    1 AV   
                                   31207  H007  R248  00-03-01    1 AV   
                                   31208  H007  R248  00-03-01    1 AV   
                        2019-05-01 31209  H007  R248  00-03-01    1 AV   
                                   31210  H007  R248  00-03-01    1 AV   
                                   31211  H007  R248  00-03-01    1 AV   
                                   31212  H007  R248  00-03-01    1 AV   
                                   31213  H007  R248  00-03-01    1 AV   
                                   31214  H007  R248  00-03-01    1 AV   
                        2019-05-02 31215  H007  R248  00-03-01    1 AV   
                                   31216  H007  R248  00-03-01    1 AV   
                                   31217  H007  R248  00-03-01    1 AV   
                                   31218  H007  R248  00-03-01    1 AV   
                                   31219  H007  R248  00-03-01    1 AV   
                                   31220  H007  R248  00-03-01    1 AV   
                                   31221  H007  R248  00-03-01    1 AV   
                        2019-05-03 31222  H007  R248  00-03-01    1 AV   
                                   31223  H007  R248  00-03-01    1 AV   
                                   31224  H007  R248  00-03-01    1 AV   
                                   31225  H007  R248  00-03-01    1 AV   
                                   31226  H007  R248  00-03-01    1 AV   
                                   31227  H007  R248  00-03-01    1 AV   
                                   31228  H007  R248  00-03-01    1 AV   

                                         LINENAME DIVISION       DATE  \
STATLINE BOOTH SCP      DATE                                            
1 AV-L   H007  00-03-01 2019-04-27 31184        L      BMT 2019-04-27   
                                   31185        L      BMT 2019-04-27   
                                   31186        L      BMT 2019-04-27   
                                   31187        L      BMT 2019-04-27   
                                   31188        L      BMT 2019-04-27   
                                   31189        L  

In [28]:
grouped_entry = grouped[(grouped.entry_diff > 0)]
grouped_exit = grouped[(grouped.exit_diff > 0)]


STATLINE     BOOTH  SCP       DATE              
1 AV-L       H007   00-00-00  2019-04-27  31050     5
                                          31051     5
                                          31052     5
                                          31053     5
                                          31054     5
                                                   ..
ZEREGA AV-6  R419   00-03-01  2019-05-03  181576    4
                                          181577    4
                                          181578    4
                                          181579    4
                    00-05-01  2019-04-27  181622    5
Name: DAY, Length: 174379, dtype: int64

In [26]:
#d.to_datetime(grouped_entry['DATE'], format= "%m/%d/%Y").dt.date
#pd.to_datetime(grouped_exit['DATE'], format= "%m/%d/%Y").dt.date

BOOTH  UNIT       SCP    STATION  \
STATLINE    BOOTH SCP      DATE                                                 
1 AV-L      H007  00-00-00 2019-04-27 31050   H007  R248  00-00-00       1 AV   
                                      31051   H007  R248  00-00-00       1 AV   
                                      31052   H007  R248  00-00-00       1 AV   
                                      31053   H007  R248  00-00-00       1 AV   
                                      31054   H007  R248  00-00-00       1 AV   
...                                            ...   ...       ...        ...   
ZEREGA AV-6 R419  00-03-01 2019-05-03 181576  R419  R326  00-03-01  ZEREGA AV   
                                      181577  R419  R326  00-03-01  ZEREGA AV   
                                      181578  R419  R326  00-03-01  ZEREGA AV   
                                      181579  R419  R326  00-03-01  ZEREGA AV   
                  00-05-01 2019-04-27 181622  R419  R326  00-05-01  ZEREGA AV   

                                             LINENAME DIVISION       DATE  \
STATLINE    BOOTH SCP      DATE                                             
1 AV-L      H007  00-00-00 2019-04-27 31050         L      BMT 2019-04-27   
                                      31051         L      BMT 2019-04-27   
                                      31052         L      BMT 2019-04-27   
                                      31053         L      BMT 2019-04-27   
                                      31054         L      BMT 2019-04-27   
...                                               ...      ...        ...   
ZEREGA AV-6 R419  00-03-01 2019-05-03 181576        6      IRT 2019-05-03   
                                      181577        6      IRT 2019-05-03   
                                      181578        6      IRT 2019-05-03   
                                      181579        6      IRT 2019-05-03   
                  00-05-01 2019-04-27 181622        6      IRT 2019-04-27   

                                                            TIME     DESC  \
STATLINE    BOOTH SCP      DATE                                             
1 AV-L      H007  00-00-00 2019-04-27 31050  1900-01-01 04:00:00  REGULAR   
                                      31051  1900-01-01 08:00:00  REGULAR   
                                      31052  1900-01-01 12:00:00  REGULAR   
                                      31053  1900-01-01 16:00:00  REGULAR   
                                      31054  1900-01-01 20:00:00  REGULAR   
...                                                          ...      ...   
ZEREGA AV-6 R419  00-03-01 2019-05-03 181576 1900-01-01 09:00:00  REGULAR   
                                      181577 1900-01-01 13:00:00  REGULAR   
                                      181578 1900-01-01 17:00:00  REGULAR   
                                      181579 1900-01-01 21:00:00  REGULAR   
                  00-05-01 2019-04-27 181622 1900-01-01 01:00:00  REGULAR   

                                               ENTRIES     EXITS  DAY  \
STATLINE    BOOTH SCP      DATE                                         
1 AV-L      H007  00-00-00 2019-04-27 31050   14647394  16373709    5   
                                      31051   14647395  16373732    5   
                                      31052   14647395  16373766    5   
                                      31053   14647395  16373800    5   
                                      31054   14647395  16373826    5   
...                                                ...       ...  ...   
ZEREGA AV-6 R419  00-03-01 2019-05-03 181576    778685    138928    4   
                                      181577    778903    138963    4   
                                      181578    779148    139017    4   
                                      181579    779273    139065    4   
                  00-05-01 2019-04-27 181622        39       145    5   

                                     

In [33]:
import datetime
start_time = datetime.datetime.strptime('07:00:00', '%H:%M:%S')
start_time.time

end_time = datetime.datetime.strptime('10:00:00', '%H:%M:%S')
end_time.time

grouped_exit2 = grouped_exit[(grouped_exit.DAY >= 0) & (grouped_exit.DAY < 5) & (grouped_exit.TIME >= start_time ) & ((grouped_exit.TIME <= end_time ))]
grouped_exit2.groupby(grouped_exit2.STATLINE).exit_diff.sum().sort_values(ascending = False).head(15)

STATLINE
TWENTY THIRD ST-1               2758258.0
GRD CNTRL-42 ST-4567S            163588.0
5 AV/53 ST-EM                     92970.0
14 ST-UNION SQ-LNQR456W           77971.0
FULTON ST-2345ACJZ                71567.0
PATH NEW WTC-1                    67756.0
47-50 STS ROCK-BDFM               60516.0
34 ST-HERALD SQ-BDFMNQRW          58538.0
42 ST-PORT AUTH-ACENQRS1237W      41208.0
72 ST-2 AVE-Q                     40890.0
34 ST-PENN STA-123ACE             40534.0
WALL ST-45                        38587.0
LEXINGTON AV/53-EM6               37351.0
W 4 ST-WASH SQ-ABCDEFM            36516.0
TIMES SQ-42 ST-1237ACENQRSW       36497.0
Name: exit_diff, dtype: float64

In [34]:
grouped_exit2 = grouped_exit[(grouped_exit.DAY > 5) & (grouped_exit.TIME >= start_time ) & ((grouped_exit.TIME <= end_time ))]
grouped_exit2.groupby(grouped_exit2.STATLINE).exit_diff.sum().sort_values(ascending = False).head(15)

STATLINE
TWENTY THIRD ST-1               1372678.0
GRD CNTRL-42 ST-4567S              2914.0
34 ST-PENN STA-123ACE              2430.0
42 ST-PORT AUTH-ACENQRS1237W       2397.0
34 ST-HERALD SQ-BDFMNQRW           2375.0
14 ST-UNION SQ-LNQR456W            2101.0
34 ST-PENN STA-ACE                 1767.0
125 ST-456                         1621.0
72 ST-123                          1574.0
TIMES SQ-42 ST-1237ACENQRSW        1494.0
59 ST COLUMBUS-ABCD1               1485.0
96 ST-123                          1484.0
14 ST-UNION SQ-456LNQRW            1458.0
CANAL ST-JNQRZ6W                   1432.0
FLUSHING-MAIN-7                    1383.0
Name: exit_diff, dtype: float64

In [49]:
names = ['23 ST', '28 ST', '14 ST', 'UNION', '42 ST']
for name in names:
    print('\n', name, ': ' , grouped[grouped.STATION.str.contains(name) ].STATLINE.unique())
    


 23 ST :  ['23 ST-1' '23 ST-6' '23 ST-CE' '23 ST-FM' '23 ST-NRW'
 'COURT SQ-23 ST-EMG']

 28 ST :  ['28 ST-1' '28 ST-6' '28 ST-NRW']

 14 ST :  ['14 ST-123FLM' '14 ST-ACEL' '14 ST-FLM123' '14 ST-UNION SQ-456LNQRW'
 '14 ST-UNION SQ-LNQR456W']

 UNION :  ['14 ST-UNION SQ-456LNQRW' '14 ST-UNION SQ-LNQR456W' 'UNION ST-R']

 42 ST :  ['42 ST-BRYANT PK-BDFM7' '42 ST-PORT AUTH-ACENGRS1237W'
 '42 ST-PORT AUTH-ACENQRS1237W' 'GRD CNTRL-42 ST-4567S'
 'TIMES SQ-42 ST-1237ACENQRS' 'TIMES SQ-42 ST-1237ACENQRSW'
 'TIMES SQ-42 ST-ACENQRS1237W']


UNION Square 14th: 14 ST-UNION SQ
23rd street 
28th street
14th street 8th avenue
42 street times square

In [50]:
correct_names = ['23 ST-6', '28 ST-6', '14 ST-ACEL', '14 ST-UNION SQ-456LNQRW' '14 ST-UNION SQ-LNQR456W' 'UNION ST-R',  'TIMES SQ-42 ST-1237ACENQRS' ,'TIMES SQ-42 ST-1237ACENQRSW',  'TIMES SQ-42 ST-ACENQRS1237W']

How we narrow down - stations near tech firms
Show the busiest stations for stations in correct_names
Show the best time for deploying volunteers at all stations in correct_names
Next time: geocoding